In [7]:
import json
import yfinance as yf

import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()


def retorna_cotacao_acao_historica(
        ticker,
        periodo='1mo'
):
    ticker_obj = yf.Ticker(f'{ticker}.SA')
    hist = ticker_obj.history(period=periodo)['Close']
    hist.index = hist.index.strftime('%Y-%m-%d')
    hist = round(hist, 2)
    if len(hist) > 30:
        slice_size = int(len(hist) / 30)
        hist = hist.iloc[::-slice_size][::-1]
    return hist.to_json()


tools = [
    {
        'type': 'function',
        'function': {
            'name': 'retorna_cotacao_acao_historica',
            'description': 'Retorna a cotação diária histórica para uma ação da bovespa',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3" para ambev, "PETR4" para petrobras, "PRIO3" para prio etc'
                    },
                    'periodo': {
                        'type': 'string',
                        'description': 'O período que será retornado de dados históriocos \
                                        sendo "1mo" equivalente a um mês de dados, "1d" a \
                                        1 dia e "1y" a 1 ano',
                        'enum': ["1d","5d","1mo","6mo","1y","5y","10y","ytd","max"]
                    }
                }
            }
        }
    }

]

funcoes_disponiveis = {'retorna_cotacao_acao_historica': retorna_cotacao_acao_historica}


mensagens = [{'role': 'user', 'content': 'Qual é a cotação da prio agora'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    tools=tools,
    tool_choice='auto'
)



# # Importa o módulo json para trabalhar com formatação e manipulação de objetos JSON.
# import json

# # Importa o módulo yfinance que permite interagir com dados financeiros do Yahoo Finance.
# import yfinance as yf

# # Importa o módulo openai para interagir com a API do OpenAI.
# import openai

# # Importa a função load_dotenv e find_dotenv do módulo dotenv para carregar variáveis de ambiente de um arquivo .env.
# from dotenv import load_dotenv, find_dotenv

# # Carrega as variáveis de ambiente do arquivo .env, se encontrado.
# _ = load_dotenv(find_dotenv())

# # Cria uma instância do cliente OpenAI para fazer chamadas à API.
# client = openai.Client()

# # Define uma função para retornar a cotação histórica de uma ação da Bovespa.
# def retorna_cotacao_acao_historica(
#         ticker,            # O código da ação (ticker) que deseja consultar, por exemplo, "PETR4".
#         periodo='1mo'      # O período para o qual os dados históricos serão retornados. O padrão é '1mo' (um mês).
# ):
#     # Cria um objeto Ticker usando o código da ação (ticker) com sufixo '.SA' para indicar ações brasileiras.
#     ticker_obj = yf.Ticker(f'{ticker}.SA')

#     # Obtém o histórico de preços de fechamento ('Close') para o período especificado.
#     hist = ticker_obj.history(period=periodo)['Close']

#     # Converte os índices da série temporal para o formato de data 'YYYY-MM-DD'.
#     hist.index = hist.index.strftime('%Y-%m-%d')

#     # Arredonda os valores das cotações para duas casas decimais.
#     hist = round(hist, 2)

#     # Se o histórico tiver mais de 30 registros, faz um recorte dos dados para retornar apenas 30 entradas.
#     if len(hist) > 30:
#         slice_size = int(len(hist) / 30)  # Calcula o tamanho do intervalo para recorte.
#         hist = hist.iloc[::-slice_size][::-1]  # Recorta e reordena os dados.

#     # Retorna o histórico como uma string JSON.
#     return hist.to_json()

# # Cria uma lista de ferramentas disponíveis para serem usadas pelo modelo da OpenAI.
# tools = [
#     {
#         'type': 'function',  # Define o tipo de ferramenta como função.
#         'function': {
#             'name': 'retorna_cotacao_acao_historica',  # Nome da função a ser chamada.
#             'description': 'Retorna a cotação diária histórica para uma ação da bovespa',  # Descrição da função.
#             'parameters': {
#                 'type': 'object',  # Define os parâmetros da função como um objeto JSON.
#                 'properties': {
#                     'ticker': {
#                         'type': 'string',  # Tipo do parâmetro ticker é uma string.
#                         'description': 'O ticker da ação. Exemplo: "ABEV3" para ambev, "PETR4" para petrobras, "PRIO3" para prio etc'  # Descrição do parâmetro ticker.
#                     },
#                     'periodo': {
#                         'type': 'string',  # Tipo do parâmetro periodo é uma string.
#                         'description': 'O período que será retornado de dados históriocos \
#                                         sendo "1mo" equivalente a um mês de dados, "1d" a \
#                                         1 dia e "1y" a 1 ano',  # Descrição do parâmetro periodo.
#                         'enum': ["1d","5d","1mo","6mo","1y","5y","10y","ytd","max"]  # Valores permitidos para o parâmetro periodo.
#                     }
#                 }
#             }
#         }
#     }
# ]

# # Mapeia as funções disponíveis para serem chamadas pela API do OpenAI.
# funcoes_disponiveis = {'retorna_cotacao_acao_historica': retorna_cotacao_acao_historica}

# # Define a mensagem inicial que simula um pedido de cotação da ação "PRIO".
# mensagens = [{'role': 'user', 'content': 'Qual é a cotação da prio agora'}]

# # Faz uma chamada à API de chat da OpenAI usando o modelo 'gpt-3.5-turbo-0125'.
# # Passa as mensagens, o modelo, as ferramentas disponíveis e define o modo de escolha de ferramentas como 'auto'.
# resposta = client.chat.completions.create(
#     messages=mensagens,
#     model='gpt-3.5-turbo-0125',
#     tools=tools,
#     tool_choice='auto'
# )


In [8]:
tool_calls = resposta.choices[0].message.tool_calls

if tool_calls:
    mensagens.append(resposta.choices[0].message)
    for tool_call in tool_calls:
        func_name = tool_call.function.name
        function_to_call = funcoes_disponiveis[func_name]
        func_args = json.loads(tool_call.function.arguments)
        func_return = function_to_call(**func_args)
        mensagens.append({
            'tool_call_id': tool_call.id,
            'role': 'tool',
            'name': func_name,
            'content': func_return
        })
    segunda_resposta = client.chat.completions.create(
        messages=mensagens,
        model='gpt-3.5-turbo-0125',
    )
    mensagens.append(segunda_resposta.choices[0].message)


In [9]:
segunda_resposta.choices[0].message.content

'A cotação da ação da Prio (PRIO3) agora é de R$ 47,66.'

In [10]:
func_name = tool_call.function.name
function_to_call = funcoes_disponiveis[func_name]
func_args = json.loads(tool_call.function.arguments)
func_return = function_to_call(**func_args)


func_return

'{"2024-08-20":47.7}'

In [11]:
func_args

{'ticker': 'PRIO3', 'periodo': '1d'}

In [12]:
tool_call

ChatCompletionMessageToolCall(id='call_ezlA8dXEDXP8n1KrOuKuA9dO', function=Function(arguments='{"ticker":"PRIO3","periodo":"1d"}', name='retorna_cotacao_acao_historica'), type='function')